<a href="https://colab.research.google.com/github/SupplyAnalytics/Marketing_Team_Projects/blob/main/All_Subcategory_PP_DP_Duplicacy_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from tqdm import tqdm
import numpy as np
from tensorflow import keras
from keras.applications import MobileNetV2
from keras.preprocessing.image import img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

In [16]:
from google.colab import drive
import os


In [17]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd

# Path to your CSV file
pp_csv_file_path = '/content/drive/My Drive/PP_VF.csv'  # Change to your CSV file path
dp_csv_file_path = '/content/drive/My Drive/DP_VF.csv'  # Change to your CSV file path

In [23]:
# Read the Excel file into a DataFrame
pp_df = pd.read_csv(pp_csv_file_path)
dp_df = pd.read_csv(dp_csv_file_path)

In [24]:
# Load MobileNetV2 without the top (classification) layers
base_model = MobileNetV2(weights='imagenet', include_top=False)
model = keras.Model(inputs=base_model.input, outputs=base_model.output)

# Function to extract feature vectors from image URLs
def extract_feature_vector(image_url):
    # Fetch the image from the URL
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = BytesIO(response.content)  # Convert to file-like object
        img = Image.open(image_data)

        # Convert image to JPEG if not already

        if img.format != 'JPEG':
            img = img.convert("RGB")  # Convert to RGB for JPEG compatibility
            image_data = BytesIO()  # Reset the file-like object
            img.save(image_data, format='JPEG')  # Save as JPEG
            image_data.seek(0)  # Reset the pointer to the beginning

        img = img.resize((224, 224))  # MobileNetV2 requires 224x224 images
        img_array = img_to_array(img)
        if img_array.shape != (224, 224, 3):
            raise ValueError("Image has incorrect shape")
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = keras.applications.mobilenet_v2.preprocess_input(img_array)  # Preprocess for MobileNetV2
        if img_array.dtype != np.float32:
            raise ValueError("Input data type is not float32")
        feature_vector = model.predict(img_array).flatten()  # Flatten to get a 1D vector
        return feature_vector
    else:
        return None

In [25]:

# Function to process a row and extract feature vector
def process_row(row):
    variant_id = str(row['VariantId'])
    subcategory = row['SubCategory']
    image_url = row['App_Image']
    feature_vector = None
    try:
        feature_vector = extract_feature_vector(image_url)
    except Exception as e:
        print(f"Error processing image at {image_url}: {e}")
    return (variant_id, subcategory, feature_vector)

# Function to extract feature vectors for a dataframe using multithreading
def feature_vectorization(df, subcategory):
    filtered_df = df[df['SubCategory'] == subcategory]
    feature_vectors = {}

    with ThreadPoolExecutor(max_workers=50) as executor:
        futures = {executor.submit(process_row, row): row for _, row in filtered_df.iterrows()}
        for future in tqdm(as_completed(futures), desc=f"Extracting Feature Vectors for Subcategory {subcategory}", total=len(futures)):
            variant_id, _, feature_vector = future.result()
            if feature_vector is not None:
                feature_vectors[variant_id] = feature_vector

    return feature_vectors

In [26]:
pp_df['SubCategory'].nunique()

53

In [27]:
# Create a directory to save results if it doesn't exist
result_dir = '/content/drive/My Drive/PP_DP_JULY_22_Results'
os.makedirs(result_dir, exist_ok=True)

In [28]:
# List existing files and extract processed subcategories
existing_files = os.listdir(result_dir)
processed_subcategories = [re.search(r'PP_DP_Task_Result_(.*).csv', file).group(1) for file in existing_files if re.search(r'PP_DP_Task_Result_(.*).csv', file)]

In [29]:
# Filter dp_subcategories to exclude already processed ones
filtered_subcategories = [sub for sub in sorted(dp_df['SubCategory'].unique()) if sub not in processed_subcategories]

In [30]:
len(processed_subcategories)

37

In [31]:
# Count the number of variants for each subcategory
subcategory_counts = dp_df.groupby('SubCategory').size()

# Sort subcategories by count in ascending order
sorted_subcategories = subcategory_counts.sort_values().index.tolist()

# Filter sorted subcategories to only include those not processed
dp_subcategories = [sub for sub in sorted_subcategories if sub in filtered_subcategories]

print("Subcategories sorted by the count of variants (ascending):")
for subcategory in dp_subcategories:
    count = subcategory_counts[subcategory]
    print(f"Subcategory: {subcategory}, Count: {count}")


print(f"Total Subcategory Count:{len(dp_subcategories)}")

Subcategories sorted by the count of variants (ascending):
Subcategory: Synthetic Handmade Sliders, Count: 134
Subcategory: kurti with bottom & dupatta, Count: 649
Subcategory: sandals, Count: 787
Subcategory: pvc moulded bellies, Count: 1142
Subcategory: pvc moulded school shoes, Count: 1551
Subcategory: eva full moulded slippers, Count: 1795
Subcategory: eva moulded slippers, Count: 2252
Subcategory: synthetic handmade formal shoes, Count: 2505
Subcategory: pvc moulded sports shoes, Count: 3409
Subcategory: machine pasted sports shoes, Count: 3639
Subcategory: shirts, Count: 6022
Subcategory: t-shirts, Count: 6058
Subcategory: Synthetic Handmade Slippers, Count: 6401
Subcategory: pu moulded slippers, Count: 6686
Total Subcategory Count:14


In [ ]:
# Get subcategories in ascending order of DP file

for subcategory in dp_subcategories:
    if subcategory in pp_df['SubCategory'].values:
        print(f"Processing subcategory: {subcategory}")

        # Extract feature vectors for pp_df and dp_df for the current subcategory
        pp_feature_vectors = feature_vectorization(pp_df, subcategory)
        dp_feature_vectors = feature_vectorization(dp_df, subcategory)

        # List to store duplicate variants for the current subcategory
        duplicate_variants = []

        # Iterate over variant IDs in pp_feature_vectors within the same subcategory
        for pp_variant_id, pp_feature_vector in pp_feature_vectors.items():
            for dp_variant_id, dp_feature_vector in dp_feature_vectors.items():
                similarity = cosine_similarity([pp_feature_vector], [dp_feature_vector])[0][0]
                if similarity >= 0.80:
                    duplicate_variants.append({
                        'PP_VariantId': pp_variant_id,
                        'DP_VariantId': dp_variant_id,
                        'SubCategory': subcategory,
                        'Similarity': similarity
                    })

        # Save the duplicate variants for the current subcategory to a CSV file
        output_csv_path = f'{result_dir}/PP_DP_Task_Result_{subcategory}.csv'
        pd.DataFrame(duplicate_variants).to_csv(output_csv_path, index=False)
        print(f"Duplicate variants for Subcategory {subcategory} saved to: {output_csv_path}")

        # Remove processed subcategory from both DataFrames if the file is successfully saved
        if os.path.exists(output_csv_path):
            pp_df = pp_df[pp_df['SubCategory'] != subcategory]
            dp_df = dp_df[dp_df['SubCategory'] != subcategory]

            # Update dp_subcategories list to avoid re-processing
            dp_subcategories.remove(subcategory)

            print("Remaining PP DataFrame rows:", pp_df['SubCategory'].nunique())
            print("Remaining DP DataFrame rows:", dp_df['SubCategory'].nunique())
            print("Remaining PP DataFrame rows:", ', '.join(pp_df['SubCategory'].unique()))
            print("Remaining PP DataFrame rows:", ', '.join(dp_df['SubCategory'].unique()))
        else:
            print(f"Subcategory {subcategory} not found in PP DataFrame.")

Processing subcategory: Synthetic Handmade Sliders


Extracting Feature Vectors for Subcategory Synthetic Handmade Sliders:   0%|          | 0/41 [00:00<?, ?it/s]

1/1 [==============================] - 23s 23s/step


Extracting Feature Vectors for Subcategory Synthetic Handmade Sliders:   5%|▍         | 2/41 [00:29<07:47, 11.99s/it]

1/1 [==============================] - 24s 24s/step


...

1/1 [==============================] - ETA: 0s

Extracting Feature Vectors for Subcategory pvc moulded sports shoes:  46%|████▋     | 1580/3409 [11:56<20:01,  1.52it/s]

1/1 [==============================] - 7s 7s/step


Extracting Feature Vectors for Subcategory pvc moulded sports shoes:  46%|████▋     | 1581/3409 [11:57<20:01,  1.52it/s]

1/1 [==============================] - ETA: 0s